- Transformer를 활용한 챗봇에서는 SubwordTextEncoder를 사용했습니다.
- SubwordTextEncoder는 사전에 토큰을 추가하거나 업데이트하는 메서드가 없어 모델 입출력시 매번 START Token과 END Token을 추가해 주는것이 번거로워 SentencePiece로 Tokenizer를 변경하고자 합니다.
- 그 과정에서 Tokenizer에 관한 내용도 한번 더 정리합니다.

# Tokenizer
 - 입력된 문장들을 토큰이라는 작은 unit들로 나누어주는 역할을 한다.
 - 토큰들 숫자데이터로 변환해 NLP task에 사용한다.
 - 문자기반, 단어기반, 서브워드 토크나이저 등등

### 1. Word Tokenizer
 - 단어를 기준으로 토큰화를 하는 토크나이저
 - ex) "맑은 하늘에 햇빛이 비쳐요." => '맑은', '하늘에', '햇빛이', '비쳐요', '.'
 - 단점으로, 어미 변화로 인해 유사단어가 많아질 수 있고 단어를 표현하는 벡터가 다른 의미를 가질 수 있다.
 - '하늘에', '하늘을', 하늘에서', '하늘이' 등
 - 사전의 개수가 매우 많이질 수 있다.
 
### 2. Character Tokenizer
 - 개별 문자를 기준으로 토큰화를 하는 토크나이저
 - ex) "안녕하세요" => '안', '녕', '하', '세', '요'
 - OOV 발생 가능성이 낮다.
 - 단점으로 각 글자 '안', '녕', '하', '세', '요'의 벡터가 '안녕하세요'를 표현하지 못할 수 있다.

### 3. Subword Tokenizer
 - 단어를 나누어 단어안에 단어(subword)들로 토큰화하는 토크나이저
 - 빈번하게 사용되는 단어는 분할하지 않고 드물게 등장하는 단어를 의미있는 하위 단어로 분할한다.
 - ex) '경찰차', '경찰청', '경찰관' => '경찰', '#차', '#청', '#관'
 - OOV 문제를 완화
 - BPE, SentencePiece, WordPiece 등 여러 방법이 존재한다.

#### 3-1) BPE

In [4]:
# BPE 
# byte pair encoding [](https://aclanthology.org/P16-1162/)
# Neural Machine Translation of Rare Words with Subword Units
# https://arxiv.org/abs/1508.07909
import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {
    'l o w </w>' : 5,
    'l o w e r </w>' : 2,
    'n e w e s t </w>':6,
    'w i d e s t </w>':3
}
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best)

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


#### 3-2) SentencePiece
 - Subword Tokenizer를 모아둔 툴
 - GitHub : [SentencePiece Docs](https://github.com/google/sentencepiece/blob/master/python/README.md)
 - 몇가지 참고사항들 :[(.vocab file은 어떻게 사용?)](https://github.com/google/sentencepiece/issues/328), [(사용법에 관한 자세한 example)](https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb)

In [5]:
# Install SentencePiece
!pip install sentencepiece

In [24]:
# SentencePiece
import sentencepiece as spm

corpus = "data/chatbot_data.txt"
prefix = "CBot_vocab"
vocab_size = 8000

# train method creates 'm.model' and 'm.vocab'.
# 'm.vocab' is just a reference. mainly use 'm.model'
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +    # maximum sentence length
    " --pad_id=0 --pad_piece=[PAD]" +    # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" +    # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" +    # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" +    # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]")    # user defined custom token

# More detail in Docs.

In [53]:
# Load the model file
sp = spm.SentencePieceProcessor(model_file='CBot_vocab.model')

# Encode: text => id
sentences = [
    "저는 금요일에 친구와 영화를 볼 거예요.",
    "안녕하세요, 오늘은 날씨가 좋습니다."]
print(sp.encode_as_pieces(sentences[0]))
print(sp.encode_as_ids(sentences[0]))
print()

# Decode: id => text
print(sp.decode_pieces(['▁저는', '▁금', '요일', '에', '▁친구와', '▁영화', '를', '▁볼', '▁거예요', '.']))
print(sp.decode_ids([615, 784, 2069, 6945, 5179, 697, 6982, 652, 24, 7564]))

['▁저는', '▁금', '요일', '에', '▁친구와', '▁영화', '를', '▁볼', '▁거예요', '.']
[615, 784, 2069, 6945, 5179, 697, 6982, 652, 24, 7564]

저는 금요일에 친구와 영화를 볼 거예요.
저는 금요일에 친구와 영화를 볼 거예요.


In [59]:
# Vocab size
print(sp.vocab_size())

# UNK Token return 1
print(sp.piece_to_id('__언노운토큰'))
print(sp.id_to_piece(1))

8007
1
[UNK]
